# Ноутбук с экспериментами с CRF

Идеи, которые хочется реализовать:
* CRF на FastText эмбеддингах
* CRF на кастомных признаках (придуманных вручную)

In [1]:
# pip install sklearn-crfsuite

In [2]:
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
import string
import re
import fasttext
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from tqdm import tqdm

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Загрузка данных

In [3]:
path = 'new_books_prepared.csv'

df = pd.read_csv(path, index_col=0)
df['tokens'] = df.tokens.apply(ast.literal_eval)
df['labels'] = df.labels.apply(ast.literal_eval)
df = df[df.tokens.apply(len) < 200]
df_train, df_val_test = train_test_split(df, test_size=0.2, random_state=999)
df_val, df_test  = train_test_split(df_val_test, test_size=0.5, random_state=999)

# одно предложение битое
df_train = df_train[~df_train.clear_punct_lower.isna()]

df_test

,text,tokens,clear_punct_lower,labels
1341,Все это смог я различить лишь смутно и с трудо...,"[все, это, смог, я, различить, лишь, смутно, и...",все это смог я различить лишь смутно и с трудо...,"[o, o, o, o, o, o, o, o, o, ., o, o, o, o, o, ..."
25027,"Она поехала в игрушечную лавку, накупила игруш...","[она, поехала, в, игрушечную, лавку, накупила,...",она поехала в игрушечную лавку накупила игруше...,"[o, o, o, o, ,, o, o, o, o, o, ., o, o, o, ,, ..."
2585,Наконец настало утро четырнадцатого числа. пог...,"[наконец, настало, утро, четырнадцатого, числа...",наконец настало утро четырнадцатого числа пого...,"[o, o, o, o, ., o, o, o, o, o, o, o, ,, o, o, ..."
16829,"Хорошо. А почему прежде, бывало, с восьми часо...","[хорошо, а, почему, прежде, бывало, с, восьми,...",хорошо а почему прежде бывало с восьми часов в...,"[., o, o, ,, ,, o, o, o, o, o, o, o, ,, o, o, ..."
7937,"Говоря это, графиня оглянулась на дочь. Наташа...","[говоря, это, графиня, оглянулась, на, дочь, н...",говоря это графиня оглянулась на дочь наташа л...,"[o, ,, o, o, o, ., o, ,, o, o, o, o, o, o, o, ..."
...,...,...,...,...
13908,Разве на одну секунду... Я пришел за советом. ...,"[разве, на, одну, секунду, я, пришел, за, сове...",разве на одну секунду я пришел за советом я ко...,"[o, o, o, ..., o, o, o, ., ,, ,, o, o, o, ,, ,..."
21490,"План был очень хорош, но дело заключалось в то...","[план, был, очень, хорош, но, дело, заключалос...",план был очень хорош но дело заключалось в том...,"[o, o, o, ,, o, o, o, o, ,, o, o, o, o, o, o, ..."
2567,"Сохраняя, поелику возможно, равновесие, чтобы ...","[сохраняя, поелику, возможно, равновесие, чтоб...",сохраняя поелику возможно равновесие чтобы хор...,"[,, o, ,, ,, o, o, o, ,, o, o, ,, o, o, o, o, ..."
25405,"Было ли в лице Левина что-нибудь особенное, ил...","[было, ли, в, лице, левина, чтонибудь, особенн...",было ли в лице левина чтонибудь особенное или ...,"[o, o, o, o, o, o, ,, o, o, o, ,, o, o, o, o, ..."


## Некоторые функции для обработки текста

In [4]:
def clear_punct_lower(line):
    line = line.lower()
    line = re.sub('–', '', line)
    line = re.sub('—', '', line).strip()
    
    # удаляем всю пунктуацию
    line = line.translate(str.maketrans('', '', string.punctuation))
    return line

clear_punct_lower('Привет, дорогая! Как дела?')

'привет дорогая как дела'

## CRF на FastText эмбеддингах

### FastText

Обучим FastText на нашем корпусе (трейн + валидация). Сперва подготовим файл для обучения:

In [5]:
with open('fasttext_train.txt', 'w') as f:
    for sent in pd.concat([df_train, df_val])['clear_punct_lower'].values:
        f.write(sent + '\n')

In [6]:
fasttext_model = fasttext.train_unsupervised('fasttext_train.txt', 
                                             minn=2, maxn=5, dim=300)

Read 0M words
Number of words:  17410
Number of labels: 0
Progress: 100.0% words/sec/thread:   27281 lr:  0.000000 avg.loss:  2.272940 ETA:   0h 0m 0s  1.2% words/sec/thread:   25623 lr:  0.049378 avg.loss:  2.804947 ETA:   0h 0m23s 70.1% words/sec/thread:   27340 lr:  0.014973 avg.loss:  2.276072 ETA:   0h 0m 6s 81.0% words/sec/thread:   27376 lr:  0.009486 avg.loss:  2.272716 ETA:   0h 0m 4s


In [7]:
fasttext_model.get_nearest_neighbors('почему')

[(0.8312212824821472, 'поэму'),
 (0.8307714462280273, 'чему'),
 (0.7862650752067566, 'поэтому'),
 (0.7751836776733398, 'помоему'),
 (0.7691491842269897, 'ничему'),
 (0.7637339234352112, 'эпоху'),
 (0.7405359148979187, 'покажу'),
 (0.7369662523269653, 'почву'),
 (0.7219629287719727, 'почемуто'),
 (0.7218359112739563, 'поймут')]

In [8]:
fasttext_model.get_word_vector('почему')[:5]

array([-0.06120062,  0.01125632,  0.11578422,  0.05215767, -0.07034817],
      dtype=float32)

### Строим CRF на векторных представлениях

In [9]:
def get_features_fasttext(word):
    try:
         vector = fasttext_model.get_word_vector(word)
    except:
        # if the word is not in vocabulary,
        # returns zeros array
        vector = np.zeros(300,)
 
    return vector  

In [10]:
def word2features(sent, i):
    word = sent[i]
    wordembdding = get_features_fasttext(word)   ## word embedding vector 
    
    features = {}

    # here you add 300 features (one for each vector component)
    for iv,value in enumerate(wordembdding):
        features['v{}'.format(iv)] = value
    
    features['bias'] = 1
    
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


In [11]:
X_train = [sent2features(s) for s in tqdm(df_train.tokens)]
y_train = df_train.labels.tolist()

X_val = [sent2features(s) for s in tqdm(df_val.tokens)]
y_val = df_val.labels.tolist()

X_test = [sent2features(s) for s in tqdm(df_test.tokens)]
y_test = df_test.labels.tolist()

100%|███████████████████████████████████████| 2586/2586 [00:27<00:00, 93.60it/s]


In [17]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=50,
    verbose=True,
    all_possible_transitions=True
)

# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|█| 20686/20686 [03:13<00:00, 106.69it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1140
Seconds required: 26.460

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=7.85  loss=687909.75 active=1139  feature_norm=1.00
Iter 2   time=3.92  loss=598171.22 active=1048  feature_norm=1.63
Iter 3   time=3.93  loss=561452.64 active=1115  feature_norm=1.61
Iter 4   time=3.94  loss=528487.34 active=1055  feature_norm=1.96
Iter 5   time=3.95  loss=507495.94 active=1131  feature_norm=1.99
Iter 6   time=3.98  loss=496545.28 active=1129  feature_norm=2.18
Iter 7   time=3.96  loss=478908.17 active=1116  feature_norm=2.64
Iter 8   time=3.91  loss=471393.75 active=1123  feature_norm=3.03
Iter 9   time=3.94  loss=466776.80 active=1131  feature_norm=3.21
Iter 1

In [20]:
y_pred_test = crf.predict(X_test)

In [34]:
from sklearn.preprocessing import LabelEncoder

all_labels = []

for row in df_test.values:
    all_labels += row[3]
    
all_preds = []

for row in y_pred_test:
    all_preds += row
    
le = LabelEncoder()

y_true = le.fit_transform(all_labels)
y_pred = le.transform(all_preds)

y_true

array([6, 6, 6, ..., 6, 6, 2])

In [35]:
from collections import Counter
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score,\
                            f1_score, precision_score, recall_score, average_precision_score

def calc_metrics_no_proba(y_true, y_pred):
    
    print('Доля пробелов:', np.mean(np.array(y_true) == 6))
    print('Accuracy:', accuracy_score(y_true, y_pred))

    metrics = []
    metrics.append(list(dict(sorted(Counter(y_true).items())).values()))
    metrics.append(f1_score(y_true, y_pred, average=None))
    metrics.append(precision_score(y_true, y_pred, average=None, zero_division=0))
    metrics.append(recall_score(y_true, y_pred, average=None, zero_division=0))
    metrics_index = ['Count', 'F1-Score', 'Precision', 'Recall']
    df_metrics = pd.DataFrame(metrics, columns=le.classes_, index=metrics_index)
    
    return df_metrics

In [37]:
calc_metrics_no_proba(y_true, y_pred)

Доля пробелов: 0.7968587786651865
Accuracy: 0.8157718914140636


,!,",",.,...,:,?,o
Count,214.000000,13022.000000,5839.000000,164.0,214.0,310.000000,77524.000000
F1-Score,0.045045,0.046402,0.488161,0.0,0.0,0.110145,0.898919
Precision,0.625000,0.329615,0.808958,0.0,0.0,0.542857,0.821187
Recall,0.023364,0.024958,0.349546,0.0,0.0,0.061290,0.992905


In [44]:
metrics.flat_f1_score(y_test, y_pred_test,
                      average='weighted', labels=list(set(all_labels) - {'o'}))

0.1770177921125934

In [45]:
metrics.flat_f1_score(y_test, y_pred_test,
                      average='weighted', labels=list(set(all_labels)))

0.7522712510902712

## CRF на кастомных признаках